# Prediction of Carpark Availability

In [5]:
# Essential Modules

import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import glob

from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(random_state=0)

from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

from urllib.request import urlopen
import json
from pandas.io.json import json_normalize

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

from sklearn.externals import joblib
import os
import warnings
warnings.filterwarnings("ignore")

In [6]:
# def convert_to_list(object):
#     return list(object)

# Convert the string to a datetime object
def convert_to_dateTime(object):
    return datetime.strptime(object, '%Y-%m-%d %H:%M:%S')

# Extract the time only from datetime object
def takeTimeOnly(time):
    return time.strftime('%H:%M:%S')

def convertTimeToSec(time):
    h, m, s = time.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def obtainDate(ts):
    return ts.strftime('%Y-%m-%d')

def obtainHour(ts):
    return ts.strftime('%H')

def obtainYear(ts):
    return ts.strftime('%Y')

def checkDayOfWeek(ts):
    #daysInWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    count = 0
    for i in range(0, 7):
        if ts.weekday() == count:
            return count
        count+=1
        
def obtainCarParkNumIndicator(listOfCarparks, carparkNumber):
    count = 0
    for i in listOfCarparks:
        if carparkNumber == i:
            return count
        count+=1

def obtainLotTypeIndicator(lotTypes, lotType):
    count = 0
    for i in lotTypes:
        if lotType == i:
            return count
        count+=1

def WeekdayOrWeekend(ts):
    day = ts.weekday()
    if (day >= 0 and day <= 4):
        return 0
    elif (day >= 5 and day <= 6): return 1
    
def identifyCarpark(carpark):
    count = 0
    for i in allCarparks:
        if i == carpark:
            return count
        count += 1

### Extraction of Data
In this notebook, the data is extracted directly from the API URL and form a structured data frame thereafter.

In [7]:
# There are 2 sections in this file, copy each section to each cell in the jupyter notebook
#
# Section 1 will produce the HDB carpark dataframe called mainHDBdf
# Section 2 will produce the URA carpark dataframe called mainURAdf

###############################  SECTION 1 ##############################################################

mainHDBdf = None

response = urlopen("http://3.14.70.180:3001/scrappeddata/hdb")
json_data = response.read().decode('utf-8', 'replace')
d = json.loads(json_data)
lengthOfHDBDatabase = len(d)

# print("There are " + str(lengthOfHDBDatabase) + " hdb json files to be downloaded")
# userChoice = int(input("1 - Download all, 2 - Subset"))

# if (userChoice == 2):
#     startNum = int(input("Enter Start Number"))
#     endNum = int(input("Enter End Number"))

# else:
#     startNum = 1
#     endNum = lengthOfHDBDatabase

for x in range(1, lengthOfHDBDatabase + 1):
    # print("Downloading hdb json file number " + str(x) + "...")
    response = urlopen("http://3.14.70.180:3001/hdbdownloads/hdbresult" + str(x) + ".json")
    json_data = response.read().decode('utf-8', 'replace')
    d = json.loads(json_data)
    # print("Processing and fitting into dataFrame...")
    for y in range(0,len(d)):
        try:
            dn = d[y]['items'][0]['carpark_data']
            for z in range(0,len(dn)):
    #             dn[z]['total_lots'] = dn[z]['carpark_info'][0]['total_lots']
                datetimeString = dn[z]['update_datetime']
                datetimeString = datetimeString[:10] + " " + datetimeString[11:]
                dn[z]['update_datetime'] = datetimeString
                dn[z]['lotType'] = dn[z]['carpark_info'][0]['lot_type']
                dn[z]['lotsAvailable'] = dn[z]['carpark_info'][0]['lots_available']
            df = json_normalize(dn)
            df = df.drop(['carpark_info'], axis=1)
            if mainHDBdf is not None:
                mainHDBdf = pd.concat([mainHDBdf,df], axis=0)
            else:
                mainHDBdf = df

        except:
            continue

mainHDBdf.columns = ['carparkNo','lotType','lotsAvailable','timeStamp']
mainHDBdf = mainHDBdf.reset_index()

mainHDBdf['year'] = mainHDBdf['timeStamp'].apply(lambda x: convert_to_dateTime(x))
mainHDBdf['year'] = mainHDBdf['year'].apply(lambda x: obtainYear(x))
mainHDBdf = mainHDBdf[mainHDBdf['year'] >= '2020']
mainHDBdf = mainHDBdf.drop(columns = ['year'])

In [7]:
# mainHDBdf = pd.read_csv('HDBData.csv')
mainHDBdf.head()

,Unnamed: 0,index,carparkNo,timeStamp,lotType,lotsAvailable
0,0,0.0,HE12,2020-02-09 02:20:51,C,57
1,1,1.0,HLM,2020-02-09 02:20:40,C,502
2,2,2.0,RHM,2020-02-09 02:20:51,C,154
3,3,3.0,BM29,2020-02-09 02:20:35,C,91
4,4,4.0,Q81,2020-02-09 02:20:47,C,70


In [8]:
##############################  SECTION 2 ##############################################################

mainURAdf = None

response = urlopen("http://3.14.70.180:3001/scrappeddata/ura")
json_data = response.read().decode('utf-8', 'replace')
d = json.loads(json_data)
lengthOfURADatabase = len(d)

# print("There are " + str(lengthOfURADatabase) + " ura json files to be downloaded")
# userChoice = int(input("1 - Download all, 2 - Subset"))

# if (userChoice == 2):
#     startNum = int(input("Enter Start Number"))
#     endNum = int(input("Enter End Number"))

# else:
#     startNum = 1
#     endNum = lengthOfURADatabase

for x in range(1, lengthOfURADatabase + 1):
    # print("Downloading ura json file number " + str(x) + "...")
    response = urlopen("http://3.14.70.180:3001/downloads/result" + str(x) + ".json")
    json_data = response.read().decode('utf-8', 'replace')
    d = json.loads(json_data)

    for y in range(0,11):
        dn = d[y]["availabilityResults"]
        df = json_normalize(dn)
        df["timeStamp"] = d[y]["timeStamp"]
        if mainURAdf is not None:
            mainURAdf = pd.concat([mainURAdf,df], axis=0)
        else:
            mainURAdf = df

mainURAdf = mainURAdf.reset_index()
mainURAdf = mainURAdf.drop(columns = ['index', 'lotType'])
mainURAdf.head()

,Unnamed: 0,carparkNo,lotsAvailable,timeStamp
0,0,S0049,107,2020-02-05 13:34:05
1,1,P0106,29,2020-02-05 13:34:05
2,2,P0094,14,2020-02-05 13:34:05
3,3,P0048,14,2020-02-05 13:34:05
4,4,O0028,28,2020-02-05 13:34:05


In [10]:
print("----------URA----------")
print(mainURAdf.dtypes, "\n")
print("Number of data points: ", len(mainURAdf))

print("\n----------HDB----------")
print(mainHDBdf.dtypes, "\n")
print("Number of data points: ", len(mainHDBdf))

----------URA----------
Unnamed: 0        int64
carparkNo        object
lotsAvailable     int64
timeStamp        object
dtype: object 

Number of data points:  367363

----------HDB----------
Unnamed: 0         int64
index            float64
carparkNo         object
timeStamp         object
lotType           object
lotsAvailable      int64
dtype: object 

Number of data points:  26152248


### Getting the carpark numbers from the data frame

In [11]:
print("----------URA----------")
print(mainURAdf.carparkNo.unique(), "\n")
print("Total number of carpaks: ", len(mainURAdf.carparkNo.unique()), "\n")

listOfURACarparks = list(mainURAdf.carparkNo.unique())

print("----------HDB----------")
print(mainHDBdf.carparkNo.unique(), "\n")
print("Total number of carpaks: ", len(mainHDBdf.carparkNo.unique()), "\n")

listOfHDBCarparks = list(mainHDBdf.carparkNo.unique())

allCarparks = listOfURACarparks + listOfHDBCarparks
allCarparks.sort()

----------URA----------
['S0049' 'P0106' 'P0094' 'P0048' 'O0028' 'M0078' 'J0100' 'D0006' 'C0162'
 'A0021' 'S0108' 'P0096' 'N0006' 'K0082' 'H0057' 'H0015' 'G0005' 'E0027'
 'E0024' 'E0023' 'A0007' 'S0166' 'T0008' 'S0171' 'P0113' 'P0117' 'S0112'
 'T0009' 'L0116' 'L0064' 'A0046' 'L0078' 'L0123' 'R0038' 'T0141' 'T0103'
 'B0087' 'K0121' 'C0148' 'L0124' 'K0111' 'P0111' 'T0140' 'M0076' 'L0117'
 'Q0006' 'S0150' 'J0017' 'L0104' 'P0054' 'Y0019' 'D0026' 'L0125' 'L0107'
 'N0013' 'A0024' 'N0012' 'B0063' 'P0093' 'J0122' 'S0020' 'B0088' 'C0119'
 'P0109' 'D0028' 'J0092' 'S0106' 'M0088' 'Q0008' 'S0055' 'H0004'] 

Total number of carpaks:  71 

----------HDB----------
['HE12' 'HLM' 'RHM' ... 'C19M' 'Q41' 'SK53'] 

Total number of carpaks:  1834 



### Data Cleaning and Features Extraction

Cleaning of the data, and getting features from the original attributes. The original data has the following attributes, carpark number, date and time, and the carpark lot availability. Hence, with the date and time of each data point, we generate the details such as the time in seconds ('timeInSec'), the hour ('hour') and the day of the week ('dayOfWeek').

In [13]:
print("Understanding of indicator data")

print("\nDay of week\n")
daysOfWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
count = 0
for i in daysOfWeek:
    print(i, ": \t", count)
    count+=1

Understanding of indicator data

Day of week

Monday : 	 0
Tuesday : 	 1
Wednesday : 	 2
Thursday : 	 3
Friday : 	 4
Saturday : 	 5
Sunday : 	 6


In [14]:
mainURAdf['timeStamp'] = mainURAdf['timeStamp'].apply(lambda x: convert_to_dateTime(x))
mainURAdf['dayOfWeek'] = mainURAdf['timeStamp'].apply(lambda x: checkDayOfWeek(x))
# mainURAdf['IsWeekend'] = mainURAdf['timeStamp'].apply(lambda x: WeekdayOrWeekend(x))
mainURAdf['hour'] = mainURAdf['timeStamp'].apply(lambda x: obtainHour(x))
mainURAdf['timeOnly'] = mainURAdf['timeStamp'].apply(lambda x: takeTimeOnly(x))
mainURAdf['timeInSec'] = mainURAdf['timeOnly'].apply(lambda x: convertTimeToSec(x))
mainURAdf['lotsAvailable'] = mainURAdf['lotsAvailable'].apply(lambda x: int(x))
lotsURA = mainURAdf.lotsAvailable
# maindf = maindf.drop(columns = ['carparkNo', 'timeStamp', 'dayOfWeek', 'hour', 'lotType', 'timeOnly', 'index', 'lotsAvailable'])
mainURAdf = mainURAdf.drop(columns = ['Unnamed: 0', 'timeOnly', 'lotsAvailable']) # 
mainURAdf = pd.concat([mainURAdf, lotsURA], axis='columns')
mainURAdf.head()

,carparkNo,timeStamp,dayOfWeek,hour,timeInSec,lotsAvailable
0,S0049,2020-02-05 13:34:05,2,13,48845,107
1,P0106,2020-02-05 13:34:05,2,13,48845,29
2,P0094,2020-02-05 13:34:05,2,13,48845,14
3,P0048,2020-02-05 13:34:05,2,13,48845,14
4,O0028,2020-02-05 13:34:05,2,13,48845,28


In [15]:
mainHDBdf['timeStamp'] = mainHDBdf['timeStamp'].apply(lambda x: convert_to_dateTime(x))
mainHDBdf['dayOfWeek'] = mainHDBdf['timeStamp'].apply(lambda x: checkDayOfWeek(x))
mainHDBdf['hour'] = mainHDBdf['timeStamp'].apply(lambda x: obtainHour(x))
mainHDBdf['timeOnly'] = mainHDBdf['timeStamp'].apply(lambda x: takeTimeOnly(x))
mainHDBdf['timeInSec'] = mainHDBdf['timeOnly'].apply(lambda x: convertTimeToSec(x))
mainHDBdf['lotsAvailable'] = mainHDBdf['lotsAvailable'].apply(lambda x: int(x))
lotsHDB = mainHDBdf.lotsAvailable
mainHDBdf = mainHDBdf.drop(columns = ['index', 'Unnamed: 0', 'lotType', 'timeOnly', 'lotsAvailable'])
mainHDBdf = pd.concat([mainHDBdf, lotsHDB], axis='columns')
mainHDBdf.head()

,carparkNo,timeStamp,dayOfWeek,hour,timeInSec,lotsAvailable
0,HE12,2020-02-09 02:20:51,6,02,8451,57
1,HLM,2020-02-09 02:20:40,6,02,8440,502
2,RHM,2020-02-09 02:20:51,6,02,8451,154
3,BM29,2020-02-09 02:20:35,6,02,8435,91
4,Q81,2020-02-09 02:20:47,6,02,8447,70


Total number of used carparks

In [19]:
print(len(mainHDBdf.carparkNo.unique()))
print(len(mainURAdf.carparkNo.unique()))
len(mainHDBdf.carparkNo.unique()) + len(mainURAdf.carparkNo.unique())

1834
71


1905

### Pre-processing of Data

The carpark number in the data frame is encoded as it is in string type.

In [20]:
combinedDF = pd.concat([mainURAdf, mainHDBdf])
combinedDF['carparkNoIndicator'] = encode.fit_transform(combinedDF['carparkNo'])
combinedDF.head()

,carparkNo,timeStamp,dayOfWeek,hour,timeInSec,lotsAvailable,carparkNoIndicator
0,S0049,2020-02-05 13:34:05,2,13,48845,107,1140
1,P0106,2020-02-05 13:34:05,2,13,48845,29,947
2,P0094,2020-02-05 13:34:05,2,13,48845,14,945
3,P0048,2020-02-05 13:34:05,2,13,48845,14,942
4,O0028,2020-02-05 13:34:05,2,13,48845,28,941


In [21]:
print(len(combinedDF))
combinedDF.tail()

26519611


,carparkNo,timeStamp,dayOfWeek,hour,timeInSec,lotsAvailable,carparkNoIndicator
26152243,B98,2020-02-29 09:42:56,5,09,34976,203,171
26152244,U70,2020-02-29 09:48:12,5,09,35292,620,1687
26152245,SB43,2020-02-29 09:50:08,5,09,35408,132,1208
26152246,Q41,2020-02-29 09:48:18,5,09,35298,38,1109
26152247,SK53,2020-02-29 09:50:02,5,09,35402,400,1336


In [23]:
# combinedDF.to_csv("allCarparksData.csv")

In [24]:
# mainURAdf.to_csv('URAData.csv')
# mainHDBdf.to_csv('HDBData.csv')

### Apply with Decision Tree Regression

sklearn Decision Tree Regression has been chosen for the model to train the data and predict the carpark availability.

In [25]:
classifierDF = combinedDF.copy()

classifierDF = classifierDF[['carparkNoIndicator', 'timeInSec', 'dayOfWeek', 'lotsAvailable']]

X2 = classifierDF[['carparkNoIndicator', 'timeInSec', 'dayOfWeek']]
y2 = classifierDF['lotsAvailable'].values.reshape(-1,1)
DTM = regr.fit(X2, y2)

model_file = 'DT_model.sav'
pickle.dump(DTM, open(model_file, 'wb'))